In [1]:
import os
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import chromadb
from uuid import uuid4
from chromadb.utils import embedding_functions

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [35]:
# TODO: Load document 
chunk_size=512
chunk_overlap = 50

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
epub_loader = UnstructuredEPubLoader('./docs/charles-dickens_a-christmas-carol.epub')
chunks = epub_loader.load_and_split(text_splitter)
print(len(chunks))

[WARNING] Could not load translations for en-US
  data file translations/en.yaml not found
[WARNING] The term Abstract has no translation defined.



431


In [36]:
# TODO: Create embedding model
embed_model_name = "BAAI/bge-small-en-v1.5"
#embed_model_name = "all-MiniLM-L6-v2"

chroma_embed_func = embedding_functions.SentenceTransformerEmbeddingFunction(
   model_name=embed_model_name
)

In [37]:
# TODO: Prepare the chunks for inserting into Chroma
texts = [ d.page_content for d in chunks ]
text_ids = [ str(uuid4())[:8] for _ in range(len(texts))]

#print(texts)
#print(text_ids)

for i in range(10):
   print(text_ids[i], '>> ', texts[i])

6c254afd >>  Imprint

The Standard Ebooks logo.

This ebook is the product of many hours of hard work by volunteers for Standard Ebooks, and builds on the hard work of other literature lovers made possible by the public domain.

This particular ebook is based on a transcription from Project Gutenberg and on digital scans from the Internet Archive.
7f6c1e18 >>  The source text and artwork in this ebook are believed to be in the United States public domain; that is, they are believed to be free of copyright restrictions in the United States. They may still be copyrighted in other countries, so users located outside of the United States must check their local laws before using this ebook. The creators of, and contributors to, this ebook dedicate their contributions to the worldwide public domain via the terms in the CC0 1.0 Universal Public Domain Dedication. For
001d0a28 >>  CC0 1.0 Universal Public Domain Dedication. For full license information, see the Uncopyright at the end of this e

In [38]:
# TODO: Create ephemeral Chroma client and save chunks
col_name = 'epub'

# Create a chroma client 
ch_client = chromadb.Client()

In [39]:
# delete collection
try: 
   ch_client.delete_collection(col_name)
except:
   print('Done')

In [40]:
# Create epub collection: collection name and the embedding function
ch_client.create_collection(name=col_name, embedding_function=chroma_embed_func)

Collection(name=epub)

In [41]:
# Insert the documents into the collection ebpub
epub_col = ch_client.get_collection(col_name)

epub_col.add(documents=texts, ids=text_ids)

In [42]:
print(epub_col.count())

431


In [43]:
# TODO: Query collection 
query = 'Scrooge'

results = epub_col.query(
   query_texts=[ query ],
   n_results=5
)

print(results)
top_n = results['ids'][0]
print(top_n)
for i in top_n:
   doc = epub_col.get(i)
   print(doc)

{'ids': [['dc684480', 'cf8709f3', '133ffd31', 'da53e500', '15f4ab6c']], 'embeddings': None, 'documents': [['“Christmas a humbug, uncle!” said Scrooge’s nephew. “You don’t mean that, I am sure?”\n\n“I do,” said Scrooge. “Merry Christmas! What right have you to be merry? What reason have you to be merry? You’re poor enough.”\n\n“Come, then,” returned the nephew gaily. “What right have you to be dismal? What reason have you to be morose? You’re rich enough.”\n\nScrooge, having no better answer ready on the spur of the moment, said, “Bah!” again; and followed it up with “Humbug!”', '“What’s today?” cried Scrooge, calling downward to a boy in Sunday clothes, who perhaps had loitered in to look about him.\n\n“Eh?” returned the boy with all his might of wonder.\n\n“What’s today, my fine fellow?” said Scrooge.\n\n“Today!” replied the boy. “Why, Christmas Day.”\n\n“It’s Christmas Day!” said Scrooge to himself. “I haven’t missed it. The Spirits have done it all in one night. They can do anything

In [22]:
model_name = 'google/flan-t5-base'

# TODO: Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
print(model)

In [67]:
question = "What happened to Marley?"
#question = "Why does scrooge dislike Christmas?"
question = "What does Scrooge's nephew try to convince him to do in Stave One?"
question = "What is the relationship between Scrooge and Marley?"
question = "What is Scrooge's job?"
question = "Why do you think that Scrooge refused to allow more than one piece of coal to be on the burner at a time?"

results = epub_col.query(
   query_texts=[ question ],
   n_results=3
)

context = ""
for ids in results['ids'][0]:
   doc  = epub_col.get(ids)
   context = f"""
      {context}

      {doc['documents'][0].strip()}
   """

print(context)


      
      
      

      External heat and cold had little influence on Scrooge. No warmth could warm, no wintry weather chill him. No wind that blew was bitterer than he, no falling snow was more intent upon its purpose, no pelting rain less open to entreaty. Foul weather didn’t know where to have him. The heaviest rain, and snow, and hail, and sleet could boast of the advantage over him in only one respect. They often “came down” handsomely, and Scrooge never did.
   

      Scrooge was at first inclined to be surprised that the Spirit should attach importance to conversations apparently so trivial; but feeling assured that they must have some hidden purpose, he set himself to consider what it was likely to be. They could scarcely be supposed to have any bearing on the death of Jacob, his old partner, for that was Past, and this Ghost’s province was the Future. Nor could he think of anyone immediately connected with himself to whom he could apply them. But nothing doubting
   

 

In [68]:
#  ("Answer based on context:\n\n{context}\n\n{question}", "{answer}")
prompt = f""" 
Answer based on context:

{context}

{question}
""" 

print(prompt)

 
Answer based on context:


      
      
      

      External heat and cold had little influence on Scrooge. No warmth could warm, no wintry weather chill him. No wind that blew was bitterer than he, no falling snow was more intent upon its purpose, no pelting rain less open to entreaty. Foul weather didn’t know where to have him. The heaviest rain, and snow, and hail, and sleet could boast of the advantage over him in only one respect. They often “came down” handsomely, and Scrooge never did.
   

      Scrooge was at first inclined to be surprised that the Spirit should attach importance to conversations apparently so trivial; but feeling assured that they must have some hidden purpose, he set himself to consider what it was likely to be. They could scarcely be supposed to have any bearing on the death of Jacob, his old partner, for that was Past, and this Ghost’s province was the Future. Nor could he think of anyone immediately connected with himself to whom he could apply them.

In [69]:
prompt_enc = tokenizer(prompt, return_tensors='pt').input_ids
answer_enc = model.generate(prompt_enc)
answer = tokenizer.decode(answer_enc[0], skip_special_tokens=True)
print(f'Question:\n{question}\n')
print(answer)

Question:
Why do you think that Scrooge refused to allow more than one piece of coal to be on the burner at a time?

He was not sure what to do.
